In [50]:
from keras.models import Model
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras import backend as K
from keras.utils import Sequence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image
from glob import glob

from keras.datasets import cifar10

imgPath = "/mnt/ml/NOAA/color"
imgCropPath = "/mnt/ml/NOAA/crop"
irPath = "/mnt/ml/NOAA/thermal"
dataPath = "/mnt/ml/NOAA/arcticseals/data"

### Metaparameters

In [6]:
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
VALIDATION_SPLIT = 0.2

### Data generator

In [7]:
def numpy_crop(img, x1, y1, x2, y2):
    return img[y1:y2, x1:x2]

In [65]:
def GetPartitionsAndLabels():
    df = pd.read_csv("{}/training.csv".format(dataPath))
    rowcount = int(df.shape[0])
    train_count = int(rowcount * (1 - VALIDATION_SPLIT))
    test_count = rowcount - train_count
    print ("Train: {} Validation: {}".format(train_count, test_count))
    
    ids = df["hotspot_id"].astype(str).values
   
    train = ids[:train_count]
    validation = ids[train_count:]
    
    print(train.shape)
    print(validation.shape)
    
    partitions = {}
    partitions['train'] = train
    partitions['validation'] = validation

    labels = {}
    for index, row in df.iterrows():
        label = 'Anomaly'
        if (row.hotspot_type == 'Animal'):
            label = row.species_id
            
        labels[str(row.hotspot_id)] = label
    
    return partitions, labels

partitions, labels = GetPartitionsAndLabels()

Train: 5299 Validation: 1325
(5299,)
(1325,)


In [66]:
def LoadImage(id):

    # Images are named <originalName>_HOTSPOT_ID.JPG
    # ID can be "1234" which Pandas converts to "1234.0" or "1234.99"
    if (id.endswith('.0')):
        id = id[:-2]

    imageMask = "{}/*HOTSPOT_{}.JPG".format(imgCropPath, id)
    imageFiles = glob(imageMask)
    assert len(imageFiles) == 1
    
    imageFile = imageFiles[0]
    image = Image.open(imageFile)
    return np.array(image)   

LoadImage(partitions['train'][0]).shape

(512, 512, 3)

In [67]:
# From https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(512,512), n_channels=3,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = LoadImage(ID)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

### VGG16 model layout:

In [5]:
def ShowLayers(base_model):
    for i, layer in enumerate(base_model.layers):
        print (i, layer.name, layer.output_shape)

In [6]:
original_model = VGG16(weights=None, include_top=True)
ShowLayers(original_model)

0 input_1 (None, 224, 224, 3)
1 block1_conv1 (None, 224, 224, 64)
2 block1_conv2 (None, 224, 224, 64)
3 block1_pool (None, 112, 112, 64)
4 block2_conv1 (None, 112, 112, 128)
5 block2_conv2 (None, 112, 112, 128)
6 block2_pool (None, 56, 56, 128)
7 block3_conv1 (None, 56, 56, 256)
8 block3_conv2 (None, 56, 56, 256)
9 block3_conv3 (None, 56, 56, 256)
10 block3_pool (None, 28, 28, 256)
11 block4_conv1 (None, 28, 28, 512)
12 block4_conv2 (None, 28, 28, 512)
13 block4_conv3 (None, 28, 28, 512)
14 block4_pool (None, 14, 14, 512)
15 block5_conv1 (None, 14, 14, 512)
16 block5_conv2 (None, 14, 14, 512)
17 block5_conv3 (None, 14, 14, 512)
18 block5_pool (None, 7, 7, 512)
19 flatten (None, 25088)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 1000)


Removing top and last 3 layers:

In [7]:
# Input = 512x512x3
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(512,512,3)) # Will download the 560MB trained model.

# Output = ['Anomaly' 'Bearded Seal' 'Ringed Seal' 'Polar Bear' 'UNK Seal']
x = base_model.output
x = Flatten(name='flatten')(x)
x = Dense(4096, name='fc1')(x)
x = Dense(4096, name='fc2')(x)
predictions = Dense(5, activation='softmax', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=predictions)
ShowLayers(model)

0 input_2 (None, 512, 512, 3)
1 block1_conv1 (None, 512, 512, 64)
2 block1_conv2 (None, 512, 512, 64)
3 block1_pool (None, 256, 256, 64)
4 block2_conv1 (None, 256, 256, 128)
5 block2_conv2 (None, 256, 256, 128)
6 block2_pool (None, 128, 128, 128)
7 block3_conv1 (None, 128, 128, 256)
8 block3_conv2 (None, 128, 128, 256)
9 block3_conv3 (None, 128, 128, 256)
10 block3_pool (None, 64, 64, 256)
11 block4_conv1 (None, 64, 64, 512)
12 block4_conv2 (None, 64, 64, 512)
13 block4_conv3 (None, 64, 64, 512)
14 block4_pool (None, 32, 32, 512)
15 block5_conv1 (None, 32, 32, 512)
16 block5_conv2 (None, 32, 32, 512)
17 block5_conv3 (None, 32, 32, 512)
18 block5_pool (None, 16, 16, 512)
19 flatten (None, 131072)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 5)


### Freeze VGG layers

In [9]:
for layer in base_model.layers: layer.trainable = False

### Compile the model

In [10]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [ ]:
params = {'dim': (512,512),
          'n_channels': 3,
          'batch_size': 64,
          'n_classes': 5,
          'shuffle': False}

training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)